# 四、对极几何

<p>
<img width=500 src="./images/对极约束.png"/>
</p>

## 求解本质矩阵

![](./images/%E5%AF%B9%E6%9E%81%E5%87%A0%E4%BD%95.png)

In [1]:
import numpy as np
from gray_video import gray_video
from core import orb_detector
from skimage.measure import ransac
from skimage.transform import EssentialMatrixTransform

In [2]:
# 相机内参
K = np.array([[1.20981082e+03, -6.38606499e-01,  6.27897309e+02],
              [0.00000000e+00,  1.18458039e+03,  3.30943028e+02],
              [0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
print("相机内参为：\n{}".format(K))


相机内参为：
[[ 1.20981082e+03 -6.38606499e-01  6.27897309e+02]
 [ 0.00000000e+00  1.18458039e+03  3.30943028e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [3]:
# 理论复现方法
def resolve_essential(matched_points: np.ndarray, K: np.ndarray) -> np.ndarray:
    K_inv = np.linalg.inv(K)
    homogeneous_coordinates = np.ones((len(matched_points), 2, 3))
    homogeneous_coordinates[:, :, :2] = matched_points
    # 使用相机内参对角点坐标归一化
    norm_curr_kps = (K_inv @ homogeneous_coordinates[:, 0].T)[:2].T
    norm_last_kps = (K_inv @ homogeneous_coordinates[:, 1].T)[:2].T
    # 求解本质矩阵和内点数据
    # 使用随机采样一致性
    model, inliers = ransac((norm_curr_kps, norm_last_kps),
                            EssentialMatrixTransform,
                            min_samples=8,              # 最少需要 8 个点
                            residual_threshold=0.001,
                            max_trials=5000)
    E = model.params
    # 返回本质矩阵
    return E


In [4]:
import cv2


def cv2_resolve_essential(img1, img2, K):
    # opencv封装好的方法
    orb = cv2.ORB_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts2 = np.float32(pts2)
    pts1 = np.float32(pts1)

    E, mask = cv2.findEssentialMat(pts1, pts2, K)
    return E


In [ ]:
matched_points = orb_detector.cross_detect(gray_video[0], gray_video[100])
E_0 = resolve_essential(matched_points, K)
E_1 = cv2_resolve_essential(gray_video[0], gray_video[100], K)


In [ ]:
E_0 /= E_0[-1, -1]
E_1 /= E_1[-1, -1]
print(E_0)
print(E_1)

[[   0.90507657  183.31556626    9.79122527]
 [-182.96078304    1.10641998  -25.10360221]
 [  -2.04065181   24.28549681    1.        ]]
[[   2.24475872  269.28355373   29.96756523]
 [-269.93929493    3.53149948  -34.68363781]
 [ -20.61488766   33.37890993    1.        ]]


可以看到使用理论推导计算出的本质矩阵与**opencv**封装之后的函数计算出的本质矩阵有些许差别，这些差别来源于：
1. 特征点的选取不同
2. 重复采样一致性原理本身具有的不确定性
3. 优化算法迭代次数的不同
但是两者具体的数值分布具有一致性。

## 从本质矩阵恢复相机位姿

从本质矩阵恢复相机位姿一般用本质矩阵的SVD分解得到。

$$
E=U\Sigma{}V^T
$$

设:

$$
W=\begin{bmatrix}
    0 & -1 & 0 \\
    1 & 0 & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
$$

有：

$$
\begin{array}{l}
     R_1 =& UWV^T\\
     R_2 =& UW^TV^T
\end{array}
$$

$t$为特征值分解后最小特征值对应的$U$矩阵中向量，其中$U$、$V$需正定

In [ ]:
def decompose_E(E: np.ndarray):
    W = np.array(
        [[0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]]
    )
    U, S, Vt = np.linalg.svd(E)
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt
    if np.linalg.det(R1) < 0:
        R1 = -R1
    if np.linalg.det(R2) < 0:
        R2 = -R2
    t = U.T[-1]
    return R1,R2,t

In [ ]:
R1, R2, t = decompose_E(E_0)
print("候选旋转矩阵1：\n{}".format(R1))
print("候选旋转矩阵2：\n{}".format(R2))
print("候选平移向量：\n{}".format(t))


候选旋转矩阵1：
[[ 0.99998472 -0.00290904  0.00470158]
 [ 0.00271275  0.99914604  0.04122909]
 [-0.0048175  -0.0412157   0.99913866]]
候选旋转矩阵2：
[[-9.64268460e-01  1.66040038e-02 -2.64406209e-01]
 [ 4.72519070e-04 -9.97924683e-01 -6.43902446e-02]
 [-2.64926618e-01 -6.22144190e-02  9.62259452e-01]]
候选平移向量：
[-0.13125328 -0.01170552  0.99127976]


## 选择正确的旋转矩阵与平移向量

用计算后的位姿转换矩阵将匹配点重投影到世界坐标中，如果所有的坐标深度都为正，那这个位姿转换矩阵所对应的旋转矩阵与平移向量则最终确定为正确的旋转矩阵与平移向量。

### 求解匹配点深度

$$
{}^ip^\land{}{}^ip={}^ip^{\land}KT{}^wP=0 \\
{}^ip^\land=\begin{bmatrix}
    0 & -1 & v \\
    1 & 0 & -u \\
    -v & u & 0 \\
\end{bmatrix}
$$

设$KT=G$，存在约束：

$$
\begin{bmatrix}
    0 & -1 & v \\
    1 & 0 & -u \\
    -v & u & 0 \\
\end{bmatrix}G{}^wP=0
$$

${}^ip^\land$的秩为$2$，每个点可以为求解${}^wP$带来两个约束，使用匹配的两个点即可求得${}^wP$

In [ ]:
normalized_K = np.eye(4)
normalized_K[:3,:3] = K
normalized_K_inv = np.linalg.inv(normalized_K)

#### 使用三角测量计算深度

In [ ]:
def triangulate(matched_points, G1, G2):
    p3ds = np.zeros((len(matched_points), 3))
    for i in range(len(matched_points)):
        point1 = matched_points[i][0]
        point2 = matched_points[i][1]
        A = np.zeros((4, 4))
        A[0] = point1[0] * G1[2] - G1[0]
        A[1] = point1[1] * G1[2] - G1[1]
        A[2] = point2[0] * G2[2] - G2[0]
        A[3] = point2[1] * G2[2] - G2[1]
        U, S, Vt = np.linalg.svd(A)
        p4d = Vt[-1]
        p3ds[i] = (p4d / p4d[-1])[:3].T
    return p3ds

#### 验证R、T

In [ ]:
def generate_G(R,t,K):
    T = np.zeros((3, 4))
    T[:,:3]= R
    T[:,3]= t
    G = K@T
    return G

In [ ]:
def check_Rt(R, t, K, matched_points):
    # 第一个相机的位姿
    R1 = np.eye(3)
    t1 = np.zeros((3))
    G1 = generate_G(R1, t1, K)
    # 第二个相机的位姿
    G2 = generate_G(R, t, K)
    # 三角测量算出匹配点在第一个相机下的世界坐标
    p3ds1 = triangulate(matched_points, G1, G2)
    # 计算匹配点在第二个相机下的世界坐标
    p3ds2 = (R @ p3ds1.T).T + t

    # 第一个相机下的匹配点世界坐标深度大于个数
    first_good_num = (p3ds1[:, -1] < 0).sum()
    # 第二个相机下的匹配点世界坐标深度是否大部分都大于0
    second_good_num = (p3ds2[:, -1] < 0).sum()

    return (first_good_num, second_good_num)


In [ ]:
print("匹配点总对数为：")
print(len(matched_points))


匹配点总对数为：
148


In [ ]:
result = check_Rt(R1, t, K, matched_points)
print("对于R1,t计算出深度为负的个数为：")
print(result)


对于R1,t计算出深度为负的个数为：
(146, 144)


In [ ]:
result = check_Rt(R1, -t, K, matched_points)
print("对于R1,-t计算出深度为负的个数为：")
print(result)


对于R1,-t计算出深度为负的个数为：
(2, 4)


In [ ]:
result = check_Rt(R2, t, K, matched_points)
print("对于R2,t计算出深度为负的个数为：")
print(result)


对于R2,t计算出深度为负的个数为：
(147, 1)


In [ ]:
result = check_Rt(R2, -t, K, matched_points)
print("对于R2,-t计算出深度为负的个数为：")
print(result)


对于R2,-t计算出深度为负的个数为：
(1, 147)


对于$R_1$、$-t$来说，计算得出世界坐标下点值为负数的个数最少，因此选择这一对参数作为相机外参。

In [ ]:
print("R1为：")
print(R1)
print("-t为：")
print(-t)


R1为：
[[ 0.99998472 -0.00290904  0.00470158]
 [ 0.00271275  0.99914604  0.04122909]
 [-0.0048175  -0.0412157   0.99913866]]
-t为：
[ 0.13125328  0.01170552 -0.99127976]
